# 텍스트 이진 분류 (긍정 or 부정)

## 0. 코랩 연결 끊김 방지

```
function ClickConnect(){
  console.log("Connnect Clicked - Start"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
  console.log("Connnect Clicked - End"); 
};
var clickid = setInterval(ClickConnect, 60*1000)
```

```
clearInterval(clickid);
```

## 1. Colab 환경 설정

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## 2. 라이브러리 설치

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/sentiment-classification

/content/drive/MyDrive/Colab Notebooks/sentiment-classification


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=689006 sha256=d2604cca9518dc58f3016f0a362a359b7ad74cd88ccb4c0e9425fb0f6c0d15bf
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 3. 라이브러리 준비

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

## 4. 데이터 준비

In [ ]:
!git clone https://github.com/e9t/nsmc.git
!git clone https://github.com/bab2min/corpus.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 8.09 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Updating files: 100% (14737/14737), done.
Cloning into 'corpus'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 2), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (15/15), 11.78 MiB | 5.88 MiB/s, done.


In [ ]:
import pandas as pd

naver_movie_train_data = pd.read_table("nsmc/ratings_train.txt")
naver_movie_test_data = pd.read_table("nsmc/ratings_test.txt")
naver_shopping_data = pd.read_table("corpus/sentiment/naver_shopping.txt")
steam_data = pd.read_table("corpus/sentiment/steam.txt")

## 5. 데이터 전처리

In [ ]:
naver_movie_train_data = naver_movie_train_data.drop(['id'], axis=1)
naver_movie_test_data = naver_movie_test_data.drop(['id'], axis=1)

naver_shopping_data.columns = ['label', 'document']
naver_shopping_data.loc[naver_shopping_data['label'] == 1, 'label'] = 0
naver_shopping_data.loc[naver_shopping_data['label'] == 2, 'label'] = 0
naver_shopping_data.loc[naver_shopping_data['label'] == 4, 'label'] = 1
naver_shopping_data.loc[naver_shopping_data['label'] == 5, 'label'] = 1

steam_data.columns = ['label', 'document']

In [ ]:
naver_movie_train_data.drop_duplicates(['document'], inplace=True)
naver_movie_test_data.drop_duplicates(['document'], inplace=True)
naver_shopping_data.drop_duplicates(['document'], inplace=True)
steam_data.drop_duplicates(['document'], inplace=True)

In [ ]:
naver_movie_train_data = naver_movie_train_data.dropna()
naver_movie_test_data = naver_movie_test_data.dropna()

In [ ]:
import numpy as np

# 한글이 아닌 것 지우기
naver_movie_train_data['document'] = naver_movie_train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
naver_movie_test_data['document'] = naver_movie_test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
naver_shopping_data['document'] = naver_shopping_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
steam_data['document'] = steam_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
# 공백 모두 지우기
naver_movie_train_data['document'] = naver_movie_train_data['document'].str.replace('^ +', "", regex=True)
naver_movie_test_data['document'] = naver_movie_test_data['document'].str.replace('^ +', "", regex=True)
naver_shopping_data['document'] = naver_shopping_data['document'].str.replace('^ +', "", regex=True)
steam_data['document'] = steam_data['document'].str.replace('^ +', "", regex=True)
# 빈 값 Nan으로 채우기
naver_movie_train_data['document'].replace('', np.nan, inplace=True)
naver_movie_test_data['document'].replace('', np.nan, inplace=True)
naver_shopping_data['document'].replace('', np.nan, inplace=True)
steam_data['document'].replace('', np.nan, inplace=True)
# Nan값 제거
naver_movie_train_data = naver_movie_train_data.dropna()
naver_movie_test_data = naver_movie_test_data.dropna()
naver_shopping_data = naver_shopping_data.dropna()
steam_data = steam_data.dropna()

## 6. train, test 세트 분리

In [ ]:
from sklearn.model_selection import train_test_split
naver_shopping_train_data, naver_shopping_test_data = train_test_split(naver_shopping_data, test_size=0.25)
steam_train_data, steam_test_data = train_test_split(steam_data, test_size=0.25)

In [ ]:
train_data = pd.concat([naver_movie_train_data, naver_shopping_train_data, steam_train_data])
test_data = pd.concat([naver_movie_test_data, naver_shopping_test_data, steam_test_data])

train_data = train_data.sample(frac=1).reset_index(drop=True) 
test_data = test_data.sample(frac=1).reset_index(drop=True) 

## 7. 데이터 변환

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/Colab Notebooks/sentiment-classification/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/Colab Notebooks/sentiment-classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /content/drive/MyDrive/Colab Notebooks/sentiment-classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset.itertuples()]
        self.labels = [np.int32(i[label_idx]) for i in dataset.itertuples()]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
data_train = BERTDataset(train_data, 1, 2, tok, max_len, True, False)
data_test = BERTDataset(test_data, 1, 2, tok, max_len, True, False)

In [ ]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## 8. 훈련

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-33-25f29a020e01>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/5786 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.725141704082489 train acc 0.421875
epoch 1 batch id 201 loss 0.5491241812705994 train acc 0.5677860696517413
epoch 1 batch id 401 loss 0.399789959192276 train acc 0.6792394014962594
epoch 1 batch id 601 loss 0.4310080409049988 train acc 0.7275634359400999
epoch 1 batch id 801 loss 0.3790273666381836 train acc 0.755539950062422
epoch 1 batch id 1001 loss 0.2354055494070053 train acc 0.773538961038961
epoch 1 batch id 1201 loss 0.39332205057144165 train acc 0.7867922564529559
epoch 1 batch id 1401 loss 0.3613922595977783 train acc 0.7968415417558886
epoch 1 batch id 1601 loss 0.27820929884910583 train acc 0.8046631011867583
epoch 1 batch id 1801 loss 0.36449313163757324 train acc 0.8106086896168795
epoch 1 batch id 2001 loss 0.391630083322525 train acc 0.8160372938530734
epoch 1 batch id 2201 loss 0.23289503157138824 train acc 0.8197481258518855
epoch 1 batch id 2401 loss 0.2900860905647278 train acc 0.8235956372344856
epoch 1 batch id 2601 loss 0.25071629881858

<ipython-input-33-25f29a020e01>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1935 [00:00<?, ?it/s]

...

epoch 2 test acc 0.8943090339892665


  0%|          | 0/5786 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.23126277327537537 train acc 0.90625
epoch 3 batch id 201 loss 0.1997833102941513 train acc 0.9231187810945274
epoch 3 batch id 401 loss 0.1668396294116974 train acc 0.9236284289276808
epoch 3 batch id 601 loss 0.1626114547252655 train acc 0.9226289517470881
epoch 3 batch id 801 loss 0.14048156142234802 train acc 0.9229088639200999
epoch 3 batch id 1001 loss 0.05171366408467293 train acc 0.9238417832167832
epoch 3 batch id 1201 loss 0.2868792712688446 train acc 0.9248282681099084
epoch 3 batch id 1401 loss 0.2836512327194214 train acc 0.9263472519628837
epoch 3 batch id 1601 loss 0.15900814533233643 train acc 0.9276721580262336
epoch 3 batch id 1801 loss 0.16177746653556824 train acc 0.9289196973903387
epoch 3 batch id 2001 loss 0.23814676702022552 train acc 0.9302614317841079
epoch 3 batch id 2201 loss 0.07018950581550598 train acc 0.9309262835074966
epoch 3 batch id 2401 loss 0.1258794069290161 train acc 0.9321181278633902
epoch 3 batch id 2601 loss 0.0993317

  0%|          | 0/1935 [00:00<?, ?it/s]

epoch 3 test acc 0.8958146988670245


## 9. 테스트

In [ ]:
# 테스트 문장 예측
test_sentence = '이 상품 너무 좋아요. 10년째 쓰는데 튼튼해요'
test_label = 1 # 실제 정답

In [ ]:
unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['document', 'label']])
# unseen_values = unseen_test.values
test_set = BERTDataset(unseen_test, 1, 2, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=2)

In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    print(out)

<ipython-input-76-1b23c9c645ec>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[-2.6136,  2.2304]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
result = torch.nn.functional.softmax(out)
result

<ipython-input-83-540d72ea9f6f>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  result = torch.nn.functional.softmax(out)


tensor([[0.0078, 0.9922]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

## 10. 모델 저장 및 불러오기

In [ ]:
import os
path = "/content" # /content is pretty much the root. you can choose other path in your colab workspace
os.chdir(path)

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/sentiment-classification

/content/drive/MyDrive/Colab Notebooks/sentiment-classification


In [ ]:
torch.save(model.state_dict(), "./best_weights_v1.pt")
torch.save(model, "./best_v1.pt")

In [ ]:
loaded_model = torch.load('best_v1.pt')

In [ ]:
# 테스트 문장 예측
test_sentence = '무난한데 잔고장이 많아서 참고 써야해요'
test_label = 0 # 실제 정답

In [ ]:
unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['document', 'label']])
# unseen_values = unseen_test.values
test_set = BERTDataset(unseen_test, 1, 2, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=2)

In [ ]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    out = loaded_model(token_ids, valid_length, segment_ids)
    print(out)

<ipython-input-139-fc2fc006e256>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[ 0.4705, -0.2804]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
result = torch.nn.functional.softmax(out)
result

<ipython-input-143-540d72ea9f6f>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  result = torch.nn.functional.softmax(out)


tensor([[0.6794, 0.3206]], device='cuda:0', grad_fn=<SoftmaxBackward0>)